In [9]:
import pandas as pd
from utils import get_dataset_dir

german_df = pd.read_csv(get_dataset_dir("financial_phrasebank_german"))                       
french_df = pd.read_csv(get_dataset_dir("financial_phrasebank_french"))
spanish_df = pd.read_csv(get_dataset_dir("financial_phrasebank_spanish"))


In [10]:
german_df = german_df.dropna()
french_df = french_df.dropna()
spanish_df = spanish_df.dropna()

In [11]:
german_df["lang"] = "de"
french_df["lang"] = "fr"
spanish_df["lang"] = "es"

In [12]:
final_dataset = pd.concat([german_df, french_df, spanish_df])
final_dataset.to_csv(get_dataset_dir("financial_phrasebank_multilingual"), index=False)

In [14]:
import datasets
dataset = datasets.load_dataset("csv", data_files=get_dataset_dir("financial_phrasebank_multilingual"))
dataset.push_to_hub("nojedag/financial_phrasebank_multilingual")

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/nojedag/financial_phrasebank_multilingual/commit/ba57e0436456062a0746b18a0264114a46b7b624', commit_message='Upload dataset', commit_description='', oid='ba57e0436456062a0746b18a0264114a46b7b624', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/nojedag/financial_phrasebank_multilingual', endpoint='https://huggingface.co', repo_type='dataset', repo_id='nojedag/financial_phrasebank_multilingual'), pr_revision=None, pr_num=None)